In [1]:
import pandas as pd
df = pd.read_csv('/content/sample_data/03_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [2]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.7 MB/s eta 0:00:00


In [3]:
%pip install lightbgm

ERROR: Could not find a version that satisfies the requirement lightbgm (from versions: none)
ERROR: No matching distribution found for lightbgm


In [9]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMRegressor


voting_reg = VotingRegressor(
    estimators=[
        ('lin_reg', LinearRegression()),
        ('LGBM', LGBMRegressor(n_estimators=100, random_state=42)),
        ('xgboost_01', xgb.XGBRegressor(max_depth=6, n_estimators=60, random_state=42)),
        ('xgboost_02', xgb.XGBRegressor(max_depth=8, n_estimators=40, random_state=42)),
        ('catboost_01',Catboost(iterations=10, depth=10, learning_rate=1, random_state=42)),
        ('catboost_02',Catboost(iterations=20, depth=8, learning_rate=1, random_state=42)),
    ]
)
voting_reg.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.114872 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1172
[LightGBM] [Info] Number of data points in the train set: 960000, number of used features: 44
[LightGBM] [Info] Start training from score 6.590488
0:	learn: 1.0674234	total: 614ms	remaining: 5.53s
1:	learn: 1.0607193	total: 1.23s	remaining: 4.9s
2:	learn: 1.0598742	total: 1.88s	remaining: 4.38s
3:	learn: 1.0593128	total: 2.6s	remaining: 3.91s
4:	learn: 1.0589240	total: 3.23s	remaining: 3.23s
5:	learn: 1.0582659	total: 3.92s	remaining: 2.61s
6:	learn: 1.0578656	total: 4.27s	remaining: 1.83s
7:	learn: 1.0572039	total: 4.62s	remaining: 1.16s
8:	learn: 1.0568427	total: 4.96s	remaining: 551ms
9:	learn: 1.0564089	total: 5.31s	remaining: 0us
0:	learn: 1.06822

VotingRegressor(estimators=[('lin_reg', LinearRegression()),
                            ('LGBM', LGBMRegressor(random_state=42)),
                            ('xgboost_01',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gr...
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=8,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=40,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=42, ...)),
                            ('catboost_01',
                             <catboost.core.CatBoostRegressor object at 0x7c299adf9480>),
                            ('catboost_02',
                             <catboost.core.CatBoostRegressor object at 0x7c299adf8a30>)])

In [10]:
from sklearn.metrics import root_mean_squared_error
y_pred = voting_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0517825660635662

In [11]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [12]:
y_pred = voting_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.0599146003575006

In [13]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/content/sample_data/03_test.csv')
del test_set['Premium Amount']
y_pred = voting_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('voting_reg_08.csv', index=False)